### Limpeza da base de dados de estoque 



### Descrição das Colunas - Base de Estoque

| Nome da Coluna           | Descrição                                                                                  | Exemplo                                                                 |
|--------------------------|-------------------------------------------------------------------------------------------|-------------------------------------------------------------------------|
| **Código**               | Identificador único do produto no sistema.                                                | `1`, `3`, `5`                                                          |
| **Produto**              | Nome/descrição do produto comercializado.                                                 | `"LAPIS Nº2 HB PT C/144 SERELEPE ECOLE"`, `"ACUCAR COMUM"`             |
| **Embalagem**            | Tipo de embalagem do produto (formato e quantidade por embalagem).                        | `"CX/1"` (Caixa com 1 unidade), `"KG/1"` (Kilograma unitário)          |
| **Unidade**              | Unidade de medida primária para venda ou controle.                                        | `"CX"` (Caixa), `"UN"` (Unidade), `"KG"` (Kilograma)                   |
| **Código da Marca**      | Identificador único da marca/fabricante do produto.                                       | `1` (Maria Geni), `4281` (Grupo Master)                                |
| **Marca**                | Nome da marca ou fornecedor do produto.                                                   | `"COMERCIO E IMPORTACAO SERTIC LTDA"`                           |
| **Quantidade estoque**   | Quantidade disponível em estoque (vazia indica registro sem dados).                       | `0`, `61`                                                              |
| **Preço de custo**       | Preço unitário de custo do produto (formato numérico).                                    | `9`, `1.48`, `2.5`                                                     |
| **Código da categoria**  | Identificador numérico da categoria do produto.                                           | `708` (Lápis), `199` (Açúcar)                                          |
| **Categoria**            | Nome da categoria do produto, com abreviação entre parênteses.                            | `"LAPIS ( LAP )"`,                                          |

In [232]:
import pandas as pd

In [233]:
df_estoque = pd.read_excel("bases/relatorio_produtos.xlsx")

### Propriedades da base

In [234]:
print(f"Formato dos dados {df_estoque.shape}")

Formato dos dados (37527, 10)


In [235]:
df_estoque.columns.to_list()

['Código',
 'Produto',
 'Embalagem',
 'Unidade',
 'Código da Marca',
 'Marca',
 'Quantidade estoque',
 'Preço de custo',
 'Código da categoria',
 'Categoria']

In [236]:
df_estoque.head()

,Código,Produto,Embalagem,Unidade,Código da Marca,Marca,Quantidade estoque,Preço de custo,Código da categoria,Categoria
0,1,LAPIS Nº2 HB PT C/144 SERELEPE ECOLE,CX/1,CX,1,MARIA GENI DISTRIB. DE ART. DE PAP. LTDA,0.0,9.00000,708,LAPIS ( LAP )
1,3,CANETA HIDRO PONTA FINA PT CIS,UN/1,UN,1272,COMERCIO E IMPORTACAO SERTIC LTDA,NaN,1.48000,35011,CANETA HIDROGRAFICA (CNT)
2,4,BANDEIRINHAS JUNINAS MEGA,PC/1,PC,1,MARIA GENI DISTRIB. DE ART. DE PAP. LTDA,NaN,2.73970,359,BANDEIRAS ( BDJ )
3,5,FITA IMP EPSON FX890 MASTERPRINT,UN/1,UN,4281,GRUPO MASTER COMERCIO IMPORTACAO E EXPORTACAO ...,61.0,10.54527,3,FITAS PARA IMPRESSORAS ( FTA )
4,6,ACUCAR COMUM,KG/1,KG,1,MARIA GENI DISTRIB. DE ART. DE PAP. LTDA,NaN,2.50000,199,AÇÚCAR


In [237]:
df_estoque.dtypes

Código                   int64
Produto                 object
Embalagem               object
Unidade                 object
Código da Marca          int64
Marca                   object
Quantidade estoque     float64
Preço de custo         float64
Código da categoria      int64
Categoria               object
dtype: object

In [238]:
df_estoque.describe()

,Código,Código da Marca,Quantidade estoque,Preço de custo,Código da categoria
count,37527.000000,37527.000000,31204.000000,37527.000000,37527.000000
mean,19170.185413,4116.673302,112.105966,22.896258,2661.045034
std,11023.203749,19900.846543,1310.514860,431.449346,8705.304106
min,1.000000,1.000000,-4.000000,0.000000,1.000000
25%,9616.500000,995.000000,0.000000,2.524401,83.000000
50%,19186.000000,2712.000000,0.000000,6.880567,322.000000
75%,28728.500000,7346.000000,20.000000,16.858232,647.000000
max,38230.000000,901455.000000,166112.000000,79000.000000,35157.000000


In [239]:
print("Quantidade valores Nulos")
df_estoque.isnull().sum()

Quantidade valores Nulos


Código                    0
Produto                   0
Embalagem                 0
Unidade                   0
Código da Marca           0
Marca                     0
Quantidade estoque     6323
Preço de custo            0
Código da categoria       0
Categoria                 0
dtype: int64

In [240]:
print("Quantidade de valores zerados")
colunas_numericas = df_estoque.select_dtypes(include='number')
for rotulo in colunas_numericas:
    qt_zeros = (colunas_numericas[rotulo] == 0).sum()
    print(f"\n{rotulo} contém {qt_zeros} zeros")

Quantidade de valores zerados

Código contém 0 zeros

Código da Marca contém 0 zeros

Quantidade estoque contém 17001 zeros

Preço de custo contém 2063 zeros

Código da categoria contém 0 zeros


In [241]:
print("Quantidade de valores negativos")
for rotulo in colunas_numericas:
    qt_negativos = (colunas_numericas[rotulo] < 0).sum()
    print(f"\n{rotulo} contém {qt_negativos} negativos")

Quantidade de valores negativos

Código contém 0 negativos

Código da Marca contém 0 negativos

Quantidade estoque contém 2 negativos

Preço de custo contém 0 negativos

Código da categoria contém 0 negativos


### Tratamento inicial 

In [242]:
# Na coluna Quantidade estoque contém alguns valores nulos e negativos, estes serão convertidos para '0'.
df_estoque.loc[df_estoque["Quantidade estoque"] < 0, "Quantidade estoque"] = 0
df_estoque['Quantidade estoque'] = df_estoque['Quantidade estoque'].fillna(0)

In [243]:
# Separar somente as colunas que serão usadas no modelo
df_estoque = df_estoque[['Código', 'Produto', 'Código da categoria', 'Categoria', 'Código da Marca', 'Marca', 'Preço de custo', 'Quantidade estoque']]
df_estoque.head()

,Código,Produto,Código da categoria,Categoria,Código da Marca,Marca,Preço de custo,Quantidade estoque
0,1,LAPIS Nº2 HB PT C/144 SERELEPE ECOLE,708,LAPIS ( LAP ),1,MARIA GENI DISTRIB. DE ART. DE PAP. LTDA,9.00000,0.0
1,3,CANETA HIDRO PONTA FINA PT CIS,35011,CANETA HIDROGRAFICA (CNT),1272,COMERCIO E IMPORTACAO SERTIC LTDA,1.48000,0.0
2,4,BANDEIRINHAS JUNINAS MEGA,359,BANDEIRAS ( BDJ ),1,MARIA GENI DISTRIB. DE ART. DE PAP. LTDA,2.73970,0.0
3,5,FITA IMP EPSON FX890 MASTERPRINT,3,FITAS PARA IMPRESSORAS ( FTA ),4281,GRUPO MASTER COMERCIO IMPORTACAO E EXPORTACAO ...,10.54527,61.0
4,6,ACUCAR COMUM,199,AÇÚCAR,1,MARIA GENI DISTRIB. DE ART. DE PAP. LTDA,2.50000,0.0


#### Para garantir a integridade dos dados e a eficácia do sistema de recomendação/cross-selling, é essencial verificar:

#### Unicidade dos Identificadores

Código: Cada valor deve ser único e corresponder a um único Produto.

Código da Marca: Deve ser único para cada Marca registrada.

Código da categoria: Deve ser único e vinculado a uma única Categoria.

In [244]:
df_estoque.nunique()

Código                 37527
Produto                36637
Código da categoria      747
Categoria                705
Código da Marca          892
Marca                    872
Preço de custo         25718
Quantidade estoque      1418
dtype: int64

In [245]:
codigos_produtos = df_estoque['Código'].nunique()
produtos_unicos = df_estoque['Produto'].nunique()
print(f"{codigos_produtos} Código de Produtos Únicos")
print(f"{produtos_unicos} Produtos Únicos")

37527 Código de Produtos Únicos
36637 Produtos Únicos


In [246]:
# Verificando se há códigos duplicados associados a produtos diferentes
codigos_duplicados = df_estoque[df_estoque["Código"].duplicated(keep=False)]
resultado = codigos_duplicados.sort_values("Código")
if resultado.isnull:
    print("Não exite códigos de produtos duplicados")
else:
    print("Existem códigos duplicados, erro crítico")

Não exite códigos de produtos duplicados


In [247]:
# Remover todos os produtos cuja descrição contém mais de um código 
produto_multiplos_codigos = df_estoque.groupby("Produto")["Código"].nunique()
produtos_problemas = produto_multiplos_codigos[produto_multiplos_codigos > 1].index
df_estoque = df_estoque[~df_estoque["Produto"].isin(produtos_problemas)]

In [248]:
df_estoque.nunique()

Código                 36251
Produto                36251
Código da categoria      743
Categoria                701
Código da Marca          888
Marca                    868
Preço de custo         25336
Quantidade estoque      1393
dtype: int64

In [230]:
# df_estoque.to_excel("bases/bases_limpas/base_estoque_limpo.xlsx", index=False)